In [66]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
import pickle, os
from numpy import dot
from numpy.linalg import norm
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer

root = Tk()
tabs = ttk.Notebook(root)
tabs.pack(fill=BOTH, expand=TRUE)
busqFrame = ttk.Frame(tabs)
clasiFrame = ttk.Frame(tabs)
tabs.add(busqFrame, text="Búsqueda de noticias")
tabs.add(clasiFrame, text="Comparación de noticias")
vector_df = pd.read_csv("vectores.csv")

def comparar():
    pass

ttk.Label(clasiFrame, text="Periódico").grid(column=0, row=0,sticky=N)
Med = IntVar() 
Medios = ["20Min", "ElMundo", "ElPais"]
Medios_choices =  ttk.Combobox(clasiFrame, textvariable=Med, state="readonly", values=Medios).grid(column=0, row=1, sticky=(N,W))

ttk.Label(clasiFrame, text="Categoría ").grid(column=1, row=0, sticky=N)
Categ = IntVar()
Categorias = ["Ciencia", "Salud", "Tecnología"]
Categ_choices = ttk.Combobox(clasiFrame, textvariable=Categ, state="readonly", values=Categorias).grid(column=1, row=1, sticky=N)


# Ranking
ttk.Label(clasiFrame, text="Noticias").grid(column=2, columnspan=3, row=0, sticky=W)
choices = ["1.txt", "2.txt", "3.txt"]
choicesvar = StringVar(value=choices)
listbox = Listbox(clasiFrame, listvariable=choicesvar).grid(column=2, columnspan=3, row=1, sticky=(N,E,W))


ttk.Label(clasiFrame, text="Preview de la noticia:").grid(column=2, columnspan=4, row=3, sticky=W)

Text(clasiFrame).grid(column=0, columnspan=4, row=3, sticky=(N,E,W))

min_size = 75
clasiFrame.columnconfigure(0, weight=1, minsize=min_size)
clasiFrame.columnconfigure(1, weight=1, minsize=min_size)
clasiFrame.columnconfigure(2, weight=1, minsize=min_size)
clasiFrame.columnconfigure(3, weight=1, minsize=min_size)

ttk.Label(clasiFrame, text="TOP-N:").grid(column=0, row=5, sticky=W)
# results select
top_n = IntVar()
top_n_choices = [1, 2, 3,4,5]
top_n_select = ttk.Combobox(clasiFrame, textvariable=top_n, state="readonly", values=top_n_choices).grid(column=1, row=5, sticky=W)

ttk.Label(clasiFrame, text="Filtrar:").grid(column=2, row=5, sticky=W)
# filter select
filter = IntVar()
filter_choices = ["20Min", "ElMundo", "ElPais"]
filter_select = ttk.Combobox(clasiFrame, textvariable=filter, state="readonly", values=filter_choices).grid(column=3, row=5, sticky=W)

# search Button
ttk.Button(clasiFrame, text="Buscar", default="active", command=comparar).grid(column=4, columnspan=5, row=5, sticky=E)


Text(clasiFrame).grid(column=0, columnspan=4, row=6, sticky=(N,E,W))

min_size = 75
clasiFrame.columnconfigure(0, weight=1, minsize=min_size)
clasiFrame.columnconfigure(1, weight=1, minsize=min_size)
clasiFrame.columnconfigure(2, weight=1, minsize=min_size)
clasiFrame.columnconfigure(3, weight=1, minsize=min_size)

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

def tokenize(x):
    return RegexpTokenizer(r'\w+').tokenize(x.lower())

def removeStopwords(x):
    with open("stopWords_es.txt") as f:
        text = f.read()
        prohibitedWords = text.split("\n")
        return [word for word in x if not word in prohibitedWords]

def stemming(x):
    stemmer = SnowballStemmer(language="spanish")
    return ' '.join([stemmer.stem(word) for word in x])

def calculateSimilarity(df):
    vectorizer = pickle.load(open("vectorizer.file", 'rb'))
    query = stemming(removeStopwords(tokenize(query_input.get())))
    consulta_vec = vectorizer.transform([query])
    dense = consulta_vec.todense()
    denselist = dense.tolist()
    for index, row in df.iterrows():
        vector = [float(i.strip()) for i in row["vector"][1:-1].split(",")]
        similitud = cos_sim(vector, denselist[0])
        df.loc[index, "similitud"] = similitud
    return df
                                                      
def buscar(event):
    global vector_df
    vector_df["similitud"] = 0.0
    vector_df = calculateSimilarity(vector_df)
    amount = top_n_select.current()
    sorted = vector_df.sort_values(by='similitud', ascending=False).head(top_n_choices[amount])
    found_files = []
    for index, row in sorted.iterrows():
        found_files.append(f"{row['similitud']*100:.1f}% - {row['name']}")
    choicesvar.set(found_files)

def onFileSelect(event):
    global vector_df
    selected = listbox.get(listbox.curselection()[0])
    # delete similarity before filename
    filename = selected[selected.find("-")+2::]
    row = vector_df.loc[vector_df['name'] == filename]
    path = row["path"].to_list()[0]
    if os.path.isfile(path):
        with open(path, "r") as f:
            textField.delete('1.0', END)
            textField.insert('1.0',f.read())
    else:
        messagebox.showinfo("No podemos encontrar esta noticia")
    

# Query Label
ttk.Label(busqFrame, text="Consulta:").grid(column=0, row=1, sticky=W)
                                                                                                     
# OQuery input
query = StringVar()
query_input = ttk.Entry(busqFrame, textvariable=query)
query_input.bind('<Return>', buscar)
query_input.grid(column=1, row=0, columnspan=5, sticky=(W, E))


ttk.Label(busqFrame, text="TOP-N:").grid(column=0, row=1, sticky=W)
# results select
top_n = IntVar()
top_n_choices = [3, 5, 10, 15, 20]
top_n_select = ttk.Combobox(busqFrame, textvariable=top_n, state="readonly", values=top_n_choices)
top_n_select.current(2)
top_n_select.grid(column=1, row=1, sticky=W)

ttk.Label(busqFrame, text="Filtrar:").grid(column=2, row=1, sticky=W)
# filter select
filter = IntVar()
filter_choices = ["20Min", "ElMundo", "ElPais"]
filter_select = ttk.Combobox(busqFrame, textvariable=filter, state="readonly", values=filter_choices)
filter_select.grid(column=3, row=1, sticky=W)

# search Button
ttk.Button(busqFrame, text="Buscar", default="active", command=buscar).grid(column=4, columnspan=2, row=1, sticky=E)


# Ranking
ttk.Label(busqFrame, text="Ranking:").grid(column=0, columnspan=2, row=3, sticky=W)
choices = ["1.txt", "2.txt", "3.txt"]
choicesvar = StringVar()
listbox = Listbox(busqFrame, listvariable=choicesvar)
listbox.grid(column=0, columnspan=2, row=4, sticky=(N,E,W))
listbox.bind("<<ListboxSelect>>", onFileSelect)

# Texto de la noticia
ttk.Label(busqFrame, text="Texto de la noticia:").grid(column=2, columnspan=4, row=4, sticky=W)

text_var = StringVar()
textField = Text(busqFrame)
textField.grid(column=2, columnspan=4, row=4, sticky=(N,E,W))

min_size = 75
busqFrame.columnconfigure(0, weight=1, minsize=min_size)
busqFrame.columnconfigure(1, weight=1, minsize=min_size)
busqFrame.columnconfigure(2, weight=1, minsize=min_size)
busqFrame.columnconfigure(3, weight=1, minsize=min_size)
busqFrame.columnconfigure(4, weight=1, minsize=min_size)
busqFrame.columnconfigure(5, weight=1, minsize=min_size)

for child in busqFrame.winfo_children(): 
    child.grid_configure(padx=5, pady=5)
    
for child in clasiFrame.winfo_children(): 
    child.grid_configure(padx=5, pady=5)
    

root.geometry("800x700")
root.title("Proyecto 21-22")
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)
root.mainloop()

salud.2022-01-14.004.txt
salud.2022-01-14.002.txt
salud.2021-12-02.004.txt
odio.2021-12-06.002.txt
salud.2022-01-14.002.txt


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/joelplambeck/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-66-f13447a96245>", line 116, in onFileSelect
    selected = listbox.get(listbox.curselection()[0])
IndexError: tuple index out of range


ciencia.2021-11-23.001.txt
